# 32 Bit Uniform Random Number Generator

This notebook details the interface with a 32 Bit uniform random number generator implemented in programmable logic (PL) on the Pynq-Z2. The vivado model created to connect the PS and PL can be found in Figure 1 below.<br>

![](./images/VivadoModel.PNG)
<center>_Figure 1: Vivado Model_</center><br>

The 'Uniform_0' block shown in Figure 1 above is custom IP created for this project. This IP implements the generation of the random number generation in hardware and the simulink model for this block can be found in Figure 2 below.

![](./images/SimulinkModel.PNG)
<center>_Figure 2: Simulink Model_</center><br>

In [ ]:
from pynq import Overlay
import matplotlib.pyplot as plt
import numpy as np
import time
from PIL import Image

ol = Overlay("custom_overlay.bit")    # TCL file is parsed

numVals = 10000

## Connect to Memory Map

Variables are used corresponding to the relevant memory addresses of the gateway blocks in the system generator design. The Memory Map generated by system generator can be found in Figure 3 below.<br>

![](./images/MemoryMap.JPG)
<center>_Figure 3: IP Memory Map_</center><br>

In [ ]:
seed = 0x00
enable = 0x04
load = 0x08
reset = 0x0C
rand = 0x10
values = []

In [ ]:
LFSR = ol.uniform_0

## Generation of sample values

The cells below setup the PL block to begin outputting random numbers.

In [ ]:
LFSR.write(reset, 1)                  # Reset LFSR block

In [ ]:
LFSR.write(reset, 0)                  # Unassert reset pin
LFSR.write(enable, 1)                 # Allow LFSR to operate
LFSR.write(load, 1)                   # Load seed int LFSR block
LFSR.write(seed, 5635)                # Write seed value to Din pin of LFSR

In [ ]:
LFSR.write(load, 0)                   # Allow LFSR to update current value

## Storage of generated values

The cells below capture the random numbers generated by the PL and store them in an array.

In [ ]:
start = time.time()                   # Store time before generating random numbers

for i in range(numVals):               # Store 100000 values
    values.append(LFSR.read(rand))    # append values to values array
    
process = time.time() - start         # Calculate time taken to generate required numbers
print(process)                        # Output time taken to generate required numbers

In [ ]:
len(values)                           # Output length of the values array

## Display of captured values

The cells below display the samples captured to the user in graphical form.

In [ ]:
plt.plot(values)                      # Plot time series of generated numbers
plt.xlabel('Sample (N)')
plt.ylabel('Value')
plt.title('Time Series')

## Histogram Plot

Plotting a histogram of the obtained values enables the user to easily observe uniformity of the random number generator.

In [ ]:
plt.hist(values)                       # Plot histogram of generated numbers
plt.xlabel('Value')
plt.ylabel('Occurrence')
plt.title('Histogram')